# Unsupervised Learning Trading Strategy
- Download/Load SP500 stocks prices data.
- Calculate different features and indicators on each stock.
- Aggregate on monthly level and filter top 150 most liquid stocks.
- Calculate Monthly Returns for different time-horizons.
- Download Fama-French Factors and Calculate Rolling Factor Betas.
- For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
- For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe -   ratio optimization.
- Visualize Portfolio returns and compare to SP500 returns.

# Packages Required

- pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

## Download SP500 stocks prices data

In [39]:
from statsmodels.regression.rolling import RollingOLS # A class that runs rolling (windowed) linear regressions
import pandas_datareader.data as web #Tool to pull financial and economic data (e.g., from FRED, Yahoo, IEX, etc.)
import matplotlib.pyplot as plt
#import statsmodels.api as sm #Full statsmodels library (for regression, statistical tests, time series modeling)
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import ta # To easily compute Moving Averages, RSI, MACD, Bollinger Bands
import warnings
warnings.filterwarnings('ignore')


sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].replace('.', '-')
symbols_list = list(sp500['Symbol'].unique())

end_date = '2025-01-01'
start_date = '2015-01-01'

df = yf.download(tickers = symbols_list, start=start_date, end=end_date, auto_adjust=False)
df

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price        Adj Close                                                  \
Ticker               A        AAPL        ABBV        ABNB         ABT   
Date                                                                     
2015-01-02   37.195488   24.288578   42.761036         NaN   36.744827   
2015-01-05   36.498512   23.604328   41.956314         NaN   36.752998   
2015-01-06   35.929966   23.606554   41.748642         NaN   36.335636   
2015-01-07   36.406826   23.937574   43.435959         NaN   36.630230   
2015-01-08   37.498112   24.857304   43.890247         NaN   37.383141   
...                ...         ...         ...         ...         ...   
2024-12-24  135.276016  257.578674  176.685577  134.990005  113.634544   
2024-12-26  135.007660  258.396667  175.900314  135.320007  114.139534   
2024-12-27  134.719406  254.974930  174.732224  133.384995  113.862282   
2024-12-30  133.606186  251.593079  172.955551  131.809998  111.693764   
2024-12-31  133.773468  249.817368  174.427933  131.410004  112.000725   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2015-01-02  18.539352   74.623970   72.339996   44.687176  38.306637  ...   
2015-01-05  18.428413   73.364006   71.980003   43.874546  36.981140  ...   
2015-01-06  18.469618   72.834816   70.529999   42.844669  36.252125  ...   
2015-01-07  18.577387   74.363602   71.110001   43.295250  36.797050  ...   
2015-01-08  18.900694   75.497543   72.919998   44.059597  36.259487  ...   
...               ...         ...         ...         ...        ...  ...   
2024-12-24  92.669998  358.322815  447.940002  216.161057  49.560802  ...   
2024-12-26  92.930000  357.133789  450.160004  216.131317  49.541222  ...   
2024-12-27  92.339996  352.922638  446.480011  215.070786  49.511856  ...   
2024-12-30  91.889999  349.266388  445.799988  210.679977  49.012627  ...   
2024-12-31  92.349998  348.572815  444.679993  210.580856  49.453121  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2015-01-02  209022  2426600  2228900  2534900  10220400   605900  2283466   
2015-01-05  343789  2385400  1695100  3107200  18502400  1369900  4418651   
2015-01-06  347338  3405900  1975800  4749600  16670700  1333200  5004401   
2015-01-07  348357  2872700  1472000  2833400  13590700  1038600  4554134   
2015-01-08  343147  3004500  1676600  2516800  15487500   821800  4258268   
...            ...      ...      ...      ...       ...      ...      ...   
2024-12-24  191200  1780100   692800   943900   7807000   379300   533000   
2024-12-26  258700  1736500  1218900  1394900   9652400   575700  1040900   
2024-12-27  310700  2320500  1086700  2015000  11943900   552400  1146300   
2024-12-30  320300  2914700  2180100  2642900  11080800   586800  1144600   
2024-12-31  362700  3125200  1612600  2143800  12387800   641600  1217100   

Price                                 
Ticker          ZBH    ZBRA      ZTS  
Date                                  
2015-01-02   936579  411800  1784200  
2015-01-05  2223873  420300  3112100  
2015-01-06  1835563  527500  3977200  
2015-01-07  1505860  467800  2481800  
2015-01-08  1449004  324400  3121300  
...             ...     ...      ...  
2024-12-24   458600   88700  1023600  
2024-12-26  1277300  140100  2167200  
2024-12-27   743400  287200  1800100  
2024-12-30  1532000  211300  1531400  
2024-12-31   683300  327900  1327400  

[2516 rows x 3018 columns]

In [40]:
df.fillna(0, inplace=True) #Replaced all NULL values with 0
df.isnull().sum() #Checking if still any NUll are left

Price      Ticker
Adj Close  A         0
           AAPL      0
           ABBV      0
           ABNB      0
           ABT       0
                    ..
Volume     XYL       0
           YUM       0
           ZBH       0
           ZBRA      0
           ZTS       0
Length: 3018, dtype: int64

In [41]:

df = df.stack() #Order the df date wise and not stock-wise - Formally, it creates a multi-index
df.index.names = ['Date', 'Ticker'] # After multi-indexing, the names are generic so, we rename them for our convinience

In [42]:
df.columns=df.columns.str.lower() # Converting to lowercase for better data reading

In [43]:
df
df.to_csv('sp500_stock_data.xlsx') #Converting the data frame to a csv file

# 2. Calculate features and technical indicators for each stock.
- Garman-Klass Volatility - Gives Volatility of Stocks
- RSI - Gives the momentum of the stock, whether it is being bought more or sold more
- Bollinger Bands - Measures Volatility and extreme, Looks at price deviation from moving average
- ATR - ATR tells you how much a stock typically moves per day. Similar to Garman-Klass but different methods
- MACD - MACD tells you when momentum is shifting — i.e., when trends are beginning, strengthening, or ending.
- Dollar Volume - Dollar Volume = Price × Volume. It tells you how much money is flowing through a stock in a day (or over any time window).

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [44]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2*np.log(2) - 1)*(np.log(df['adj close']) - np.log(df['open']))**2
df['rsi'] = df.groupby('Ticker')['adj close'].transform(lambda x : ta.momentum.RSIIndicator(x, window=14).rsi()) 
# Grouping the unique ticker values and extracting only the 'adj close' from each unique ticker value. Further, we 
# calculate the RSI on each using transform and lambda.

In [45]:
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi  
Date       Ticker                                            
2015-01-02 A         1529200.0         -0.003736        NaN  
           AAPL    212818400.0         -0.006523        NaN  
           ABBV      5086100.0         -0.069835        NaN  
           ABNB            0.0               NaN        NaN  
           ABT       3216600.0         -0.016584        NaN  
...                        ...               ...        ...  
2024-12-31 XYL        641600.0          0.000024  32.070978  
           YUM       1217100.0          0.000034  46.929333  
           ZBH        683300.0          0.000085  41.626049  
           ZBRA       327900.0          0.000088  43.505840  
           ZTS       1327400.0          0.000052  31.259963  

[1265548 rows x 8 columns]

In [46]:
df['bblow'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_lband())
df['bbmid'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_mavg())
df['bbhigh'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_hband())
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003736        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh  
Date       Ticker                      
2015-01-02 A            NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABNB         NaN       NaN  
           ABT          NaN       NaN  
...                     ...       ...  
2024-12-31 XYL     4.799889  4.873531  
           YUM     4.908393  4.945308  
           ZBH     4.679910  4.706731  
           ZBRA    5.989551  6.036588  
           ZTS     5.140142  5.212182  

[1265548 rows x 11 columns]

In [47]:
import ta.volatility


def computeATR(dataframe) :
    atr = ta.volatility.average_true_range(high=dataframe['high'],
                                           low=dataframe['low'],
                                           close=dataframe['close'],
                                           window=20)
    atr = (atr - atr.mean()) / atr.std() #NORMALIZING
    return atr
df['atr'] = df.groupby(level=1, group_keys=False).apply(computeATR)
#df['atr'] = df.groupby(level=1)['atr'].transform(lambda x: (x - x.mean()) / x.std())
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003736        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh       atr  
Date       Ticker                                
2015-01-02 A            NaN       NaN -1.783827  
           AAPL         NaN       NaN -1.364950  
           ABBV         NaN       NaN -2.518021  
           ABNB         NaN       NaN -0.719302  
           ABT          NaN       NaN -1.978930  
...                     ...       ...       ...  
2024-12-31 XYL     4.799889  4.873531  0.751302  
           YUM     4.908393  4.945308  0.757992  
           ZBH     4.679910  4.706731 -0.542031  
           ZBRA    5.989551  6.036588  0.300231  
           ZTS     5.140142  5.212182  0.623447  

[1265548 rows x 12 columns]

In [48]:
from ta.trend import MACD

df['macd'] = df.groupby(level=1)['adj close'].transform(lambda x : MACD(close=x, window_fast=20).macd())
df['macd'] = df.groupby(level=1)['macd'].transform(
    lambda x: (x - x.mean()) / x.std() #NORMALIZING
)
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003736        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh       atr      macd  
Date       Ticker                                          
2015-01-02 A            NaN       NaN -1.783827       NaN  
           AAPL         NaN       NaN -1.364950       NaN  
           ABBV         NaN       NaN -2.518021       NaN  
           ABNB         NaN       NaN -0.719302       NaN  
           ABT          NaN       NaN -1.978930       NaN  
...                     ...       ...       ...       ...  
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419  
           YUM     4.908393  4.945308  0.757992 -0.432492  
           ZBH     4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.140142  5.212182  0.623447 -1.900298  

[1265548 rows x 13 columns]

In [49]:
df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003736        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh       atr      macd  dollar_volume  
Date       Ticker                                                         
2015-01-02 A            NaN       NaN -1.783827       NaN      56.879340  
           AAPL         NaN       NaN -1.364950       NaN    5169.056315  
           ABBV         NaN       NaN -2.518021       NaN     217.486905  
           ABNB         NaN       NaN -0.719302       NaN       0.000000  
           ABT          NaN       NaN -1.978930       NaN     118.193411  
...                     ...       ...       ...       ...            ...  
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419      73.972052  
           YUM     4.908393  4.945308  0.757992 -0.432492     161.724855  
           ZBH     4.679910  4.706731 -0.542031 -0.272231      71.836380  
           ZBRA    5.989551  6.036588  0.300231 -0.159019     126.641538  
           ZTS     5.140142  5.212182  0.623447 -1.900298     214.896249  

[1265548 rows x 14 columns]

## 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.
- To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [50]:
df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume') #Converting daily 
# to monthly that's it!

dollar_volume
Date       Ticker               
2015-01-31 A           94.975115
           AAPL      6444.775157
           ABBV       348.751868
           ABNB         0.000000
           ABT        212.311224
...                          ...
2024-12-31 XYL        160.744519
           YUM        204.737631
           ZBH        160.263854
           ZBRA       121.307398
           ZTS        404.846222

[60360 rows x 1 columns]

In [51]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'close', 'open', 'close', 'high','low','volume']]
last_cols

['adj close',
 'garman_klass_vol',
 'rsi',
 'bblow',
 'bbmid',
 'bbhigh',
 'atr',
 'macd']

In [52]:
df.unstack()[last_cols].resample('M').last().stack('Ticker')
data = pd.concat([df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume'), df.unstack()[last_cols].resample('M').last().stack('Ticker')
], axis=1).dropna() #Took Mean of all dollar volume in a month and last value of month for others and concatenate both

In [53]:
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2015-02-28 A          102.654068   38.708611         -0.003014  65.073055   
           AAPL      6712.875477   28.651104         -0.005975  63.065433   
           ABBV       520.677098   39.557816         -0.069846  50.513231   
           ABT        225.293288   38.971397         -0.015008  64.968729   
           ACGL        24.453041   18.751719         -0.000874  46.969949   
...                          ...         ...               ...        ...   
2024-12-31 XYL        160.744519  115.293098          0.000024  32.070978   
           YUM        204.737631  132.877213          0.000034  46.929333   
           ZBH        160.263854  105.131538          0.000085  41.626049   
           ZBRA       121.307398  386.220001          0.000088  43.505840   
           ZTS        404.846222  161.892609          0.000052  31.259963   

                      bblow     bbmid    bbhigh       atr      macd  
Date       Ticker                                                    
2015-02-28 A       3.572805  3.638320  3.703835 -1.107361  0.147813  
           AAPL    3.282025  3.361759  3.441493 -0.931137  0.174612  
           ABBV    3.617033  3.677789  3.738546 -0.740117 -0.541718  
           ABT     3.617671  3.661333  3.704995 -1.048068  0.274728  
           ACGL    2.970974  2.990333  3.009693 -1.091787 -0.260423  
...                     ...       ...       ...       ...       ...  
2024-12-31 XYL     4.726247  4.799889  4.873531  0.751302 -1.723419  
           YUM     4.871477  4.908393  4.945308  0.757992 -0.432492  
           ZBH     4.653089  4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.942514  5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.068102  5.140142  5.212182  0.623447 -1.900298  

[57789 rows x 9 columns]

In [54]:
data['dollar_volume'] = (data['dollar_volume'].unstack().rolling(5*12).mean().stack('Ticker'))
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2015-02-28 A                 NaN   38.708611         -0.003014  65.073055   
           AAPL              NaN   28.651104         -0.005975  63.065433   
           ABBV              NaN   39.557816         -0.069846  50.513231   
           ABT               NaN   38.971397         -0.015008  64.968729   
           ACGL              NaN   18.751719         -0.000874  46.969949   
...                          ...         ...               ...        ...   
2024-12-31 XYL        121.846210  115.293098          0.000024  32.070978   
           YUM        200.032726  132.877213          0.000034  46.929333   
           ZBH        177.549156  105.131538          0.000085  41.626049   
           ZBRA       124.840513  386.220001          0.000088  43.505840   
           ZTS        343.347799  161.892609          0.000052  31.259963   

                      bblow     bbmid    bbhigh       atr      macd  
Date       Ticker                                                    
2015-02-28 A       3.572805  3.638320  3.703835 -1.107361  0.147813  
           AAPL    3.282025  3.361759  3.441493 -0.931137  0.174612  
           ABBV    3.617033  3.677789  3.738546 -0.740117 -0.541718  
           ABT     3.617671  3.661333  3.704995 -1.048068  0.274728  
           ACGL    2.970974  2.990333  3.009693 -1.091787 -0.260423  
...                     ...       ...       ...       ...       ...  
2024-12-31 XYL     4.726247  4.799889  4.873531  0.751302 -1.723419  
           YUM     4.871477  4.908393  4.945308  0.757992 -0.432492  
           ZBH     4.653089  4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.942514  5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.068102  5.140142  5.212182  0.623447 -1.900298  

[57789 rows x 9 columns]

In [55]:
data['vol_rank'] = data.groupby('Date')['dollar_volume'].rank(ascending=False)
data = data[data['vol_rank'] < 150].drop(['dollar_volume', 'vol_rank'], axis=1)

In [56]:
data

adj close  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                      
2020-01-31 AAPL     74.835625         -0.000833  52.961766  4.288566   
           ABBV     65.044540         -0.019604  27.244858  4.195206   
           ABT      79.416740         -0.004793  46.622784  4.347044   
           ACN     189.660172         -0.003094  45.383468  5.242825   
           ADBE    351.140015          0.000161  62.984284  5.806371   
...                       ...               ...        ...       ...   
2024-12-31 WDAY    258.029999          0.000278  43.496656  5.552833   
           WFC      69.513557         -0.000007  45.719777  4.230842   
           WMT      89.885246          0.000025  45.842701  4.508084   
           XOM     105.643356          0.000187  34.801245  4.622929   
           ZTS     161.892609          0.000052  31.259963  5.068102   

                      bbmid    bbhigh       atr      macd  
Date       Ticker                                          
2020-01-31 AAPL    4.337738  4.386909 -0.402496  0.825655  
           ABBV    4.253540  4.311873 -0.821136 -0.609667  
           ABT     4.394598  4.442152 -0.344413  0.576504  
           ACN     5.262161  5.281497 -0.461768  0.089804  
           ADBE    5.847616  5.888861 -0.528356  0.908227  
...                     ...       ...       ...       ...  
2024-12-31 WDAY    5.598519  5.644205  1.350044  0.389197  
           WFC     4.272490  4.314138  1.524281  0.115922  
           WMT     4.541688  4.575291  3.037783  1.815964  
           XOM     4.688257  4.753586  0.301786 -2.636824  
           ZTS     5.140142  5.212182  0.623447 -1.900298  

[8940 rows x 8 columns]

## 4. Calculate Monthly Returns for different time horizons as features.
- To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [57]:
def calculate_returns(df) :
    outlier_cutoff = 0.005
    lags = [1,3,6,9,12]
    for lag in lags:
        df[f"Return_{lag}m"] = df['adj close'].pct_change(lag).pipe(lambda x: x.clip(x.quantile(outlier_cutoff), x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1)
    return df

data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-02-09 A        35.801579   39.040001   39.430000   38.950001   39.230000   
           AAPL     26.701773   29.930000   29.959999   29.607500   29.637501   
           ABBV     36.275505   55.480000   56.349998   55.180000   55.669998   
           ABT      36.626678   44.520000   45.209999   44.299999   45.209999   
           ACGL     18.945066   19.923332   20.063334   19.903334   20.006666   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-02-09 A         3586100.0         -0.003156  48.440784  3.567578   
           AAPL    155559200.0         -0.004133  66.460269  3.186643   
           ABBV     22522000.0         -0.070642  28.322007  3.633306   
           ABT       6585900.0         -0.016917  48.111295  3.600365   
           ACGL      1284300.0         -0.001116  55.264187  2.968493   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh       atr      macd  dollar_volume  \
Date       Ticker                                                          
2015-02-09 A       3.600646  3.633714 -1.006147 -0.336036     128.388041   
           AAPL    3.268119  3.349596 -0.894250 -0.057301    4153.706398   
           ABBV    3.719573  3.805839 -0.254639 -0.911382     816.996925   
           ABT     3.629855  3.659344 -0.881066 -0.129465     241.219642   
           ACGL    2.985903  3.003313 -1.060630 -0.190912      24.331149   
...                     ...       ...       ...       ...            ...   
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419      73.972052   
           YUM     4.908393  4.945308  0.757992 -0.432492     161.724855   
           ZBH     4.679910  4.706731 -0.542031 -0.272231      71.836380   
           ZBRA    5.989551  6.036588  0.300231 -0.159019     126.641538   
           ZTS     5.140142  5.212182  0.623447 -1.900298     214.896249   

Price              Return_1m  Return_3m  Return_6m  Return_9m  Return_12m  
Date       Ticker                                                          
2015-02-09 A       -0.007626  -0.000597   0.005527   0.000829   -0.001291  
           AAPL     0.006642   0.001760   0.004268   0.010776    0.005601  
           ABBV    -0.024956  -0.008447  -0.013925  -0.014198   -0.010511  
           ABT     -0.021539  -0.004820  -0.000896   0.002119    0.000056  
           ACGL    -0.003999  -0.000446   0.005109  -0.000167    0.000140  
...                      ...        ...        ...        ...         ...  
2024-12-31 XYL     -0.002236  -0.006942  -0.001600  -0.004448   -0.003544  
           YUM      0.004793  -0.004530   0.002254  -0.000471   -0.002734  
           ZBH      0.002182  -0.004315  -0.001958  -0.002275   -0.000762  
           ZBRA     0.006174  -0.009010  -0.002913  -0.002495   -0.004488  
           ZTS      0.004253  -0.005243  -0.001941  -0.005221   -0.007734  

[1209531 rows x 19

In [58]:
calculate_returns(df)
df.dropna()
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.195488   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288578   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744827   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003736        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

Price                 bbmid    bbhigh       atr      macd  dollar_volume  \
Date       Ticker                                                          
2015-01-02 A            NaN       NaN -1.783827       NaN      56.879340   
           AAPL         NaN       NaN -1.364950       NaN    5169.056315   
           ABBV         NaN       NaN -2.518021       NaN     217.486905   
           ABNB         NaN       NaN -0.719302       NaN       0.000000   
           ABT          NaN       NaN -1.978930       NaN     118.193411   
...                     ...       ...       ...       ...            ...   
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419      73.972052   
           YUM     4.908393  4.945308  0.757992 -0.432492     161.724855   
           ZBH     4.679910  4.706731 -0.542031 -0.272231      71.836380   
           ZBRA    5.989551  6.036588  0.300231 -0.159019     126.641538   
           ZTS     5.140142  5.212182  0.623447 -1.900298     214.896249   

Price              Return_1m  Return_3m  Return_6m  Return_9m  Return_12m  
Date       Ticker                                                          
2015-01-02 A             NaN        NaN        NaN        NaN         NaN  
           AAPL    -0.347002        NaN        NaN        NaN         NaN  
           ABBV     0.760541        NaN        NaN        NaN         NaN  
           ABNB    -1.000000  -1.000000        NaN        NaN         NaN  
           ABT           inf   0.147972        NaN        NaN         NaN  
...                      ...        ...        ...        ...         ...  
2024-12-31 XYL      0.091343   0.103916  -0.159503   0.028047    0.043064  
           YUM      0.152517   0.263592  -0.132348   0.096530   -0.033641  
           ZBH     -0.208807  -0.001618   0.248724  -0.060163    0.058418  
           ZBRA     2.673684   0.496272   0.285207   0.018656    0.129177  
           ZTS     -0.580828   0.068051   0.161715  -0.070118    0.089340  

[1265548 rows x 19

## We will incorporate Fama-French 3-factors into our dataset

In [59]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench',start = '2015', end = '2024-12')[0].drop('RF', axis = 1)

In [60]:
factor_data.index = pd.to_datetime(factor_data.index.to_timestamp())

In [61]:
factor_data = factor_data.resample('M').last().div(100)
factor_data

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
2015-01-31,-0.0309,-0.0093,-0.0345,0.0158,-0.0164
2015-02-28,0.0614,0.0036,-0.0179,-0.0110,-0.0175
2015-03-31,-0.0109,0.0308,-0.0038,0.0007,-0.0062
2015-04-30,0.0060,-0.0301,0.0180,0.0005,-0.0062
2015-05-31,0.0138,0.0082,-0.0111,-0.0176,-0.0083
...,...,...,...,...,...
2024-08-31,0.0161,-0.0355,-0.0110,0.0075,0.0082
2024-09-30,0.0173,-0.0092,-0.0277,0.0018,-0.0029
2024-10-31,-0.0100,-0.0088,0.0086,-0.0142,0.0098


In [62]:
factor_data.index.name = 'Date'


In [63]:
factor_data = factor_data.join(data['Return_1m']).sort_index()

In [64]:
factor_data.xs('AAPL', level=1).head()

,Mkt-RF,SMB,HML,RMW,CMA,Return_1m
Date,,,,,,
2015-03-31,-0.0109,0.0308,-0.0038,0.0007,-0.0062,-0.015352
2015-04-30,0.0060,-0.0301,0.0180,0.0005,-0.0062,-0.027130
2015-06-30,-0.0154,0.0290,-0.0082,0.0035,-0.0154,0.007227
2015-07-31,0.0157,-0.0458,-0.0416,0.0031,-0.0244,-0.008744
2015-08-31,-0.0602,0.0029,0.0275,0.0066,0.0129,-0.004678


In [65]:
factor_data.xs('MSFT', level=1).head()

,Mkt-RF,SMB,HML,RMW,CMA,Return_1m
Date,,,,,,
2015-03-31,-0.0109,0.0308,-0.0038,0.0007,-0.0062,-0.007324
2015-04-30,0.0060,-0.0301,0.0180,0.0005,-0.0062,-0.008561
2015-06-30,-0.0154,0.0290,-0.0082,0.0035,-0.0154,-0.004958
2015-07-31,0.0157,-0.0458,-0.0416,0.0031,-0.0244,-0.003840
2015-08-31,-0.0602,0.0029,0.0275,0.0066,0.0129,-0.009333


### - Filter out data whose data is known for less than 10 months
It is done because, we will be calculating rolling averages for 2 years so, data with less than 10 months
will bring anomaly

In [66]:
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations>=10]
factor_data = factor_data[factor_data.index.get_level_values('Ticker').isin(valid_stocks.index)]
factor_data

Mkt-RF     SMB     HML     RMW     CMA  Return_1m
Date       Ticker                                                   
2015-03-31 A      -0.0109  0.0308 -0.0038  0.0007 -0.0062  -0.004075
           AAPL   -0.0109  0.0308 -0.0038  0.0007 -0.0062  -0.015352
           ABBV   -0.0109  0.0308 -0.0038  0.0007 -0.0062   0.005151
           ABT    -0.0109  0.0308 -0.0038  0.0007 -0.0062  -0.015931
           ACGL   -0.0109  0.0308 -0.0038  0.0007 -0.0062  -0.017230
...                   ...     ...     ...     ...     ...        ...
2024-12-31 XYL    -0.0315 -0.0383 -0.0300  0.0190 -0.0121  -0.002236
           YUM    -0.0315 -0.0383 -0.0300  0.0190 -0.0121   0.004793
           ZBH    -0.0315 -0.0383 -0.0300  0.0190 -0.0121   0.002182
           ZBRA   -0.0315 -0.0383 -0.0300  0.0190 -0.0121   0.006174
           ZTS    -0.0315 -0.0383 -0.0300  0.0190 -0.0121   0.004253

[40766 rows x 6 columns]

### Now, we will calculate betas using rolling regressions.
Rolling Regressions fit the data of a time period and estimate betas of FAMA-FRENCH factors

In [67]:
betas = (factor_data.groupby(level=1, group_keys=False).apply(lambda x : RollingOLS(x['Return_1m'], x.drop('Return_1m', axis = 1), min(24, x.shape[0]), min_nobs=len(x.columns) + 1).fit(params_only = True).params))
betas

Mkt-RF       SMB       HML       RMW       CMA
Date       Ticker                                                  
2015-03-31 A            NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACGL         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-12-31 XYL     0.137794  0.017366  0.022785  0.229996 -0.373432
           YUM     0.084030  0.055613  0.011179  0.214598 -0.120012
           ZBH     0.173074 -0.155981  0.058024  0.017543 -0.177280
           ZBRA    0.065180  0.164956 -0.072873  0.136915 -0.063067
           ZTS     0.073094  0.057430 -0.154298  0.189346  0.073562

[40766 rows x 5 columns]

In [68]:
betas.groupby('Ticker').shift()

Mkt-RF       SMB       HML       RMW       CMA
Date       Ticker                                                  
2015-03-31 A            NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACGL         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-12-31 XYL     0.137447  0.004861  0.027339  0.237713 -0.386753
           YUM     0.086861  0.058779  0.013125  0.211814 -0.121965
           ZBH     0.163961 -0.129049  0.037081  0.003908 -0.129447
           ZBRA    0.073991  0.163371 -0.062295  0.135212 -0.081946
           ZTS     0.073236  0.060556 -0.155374  0.187400  0.076784

[40766 rows x 5 columns]

In [69]:
data = data.join(betas.groupby('Ticker').shift())

In [70]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
data.loc[:, factors] = data.groupby('Ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))
data.dropna()
data

adj close       close        high         low        open  \
Date       Ticker                                                               
2015-02-09 A        35.801579   39.040001   39.430000   38.950001   39.230000   
           AAPL     26.701773   29.930000   29.959999   29.607500   29.637501   
           ABBV     36.275505   55.480000   56.349998   55.180000   55.669998   
           ABT      36.626678   44.520000   45.209999   44.299999   45.209999   
           ACGL     18.945066   19.923332   20.063334   19.903334   20.006666   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

                        volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-02-09 A         3586100.0         -0.003156  48.440784  3.567578   
           AAPL    155559200.0         -0.004133  66.460269  3.186643   
           ABBV     22522000.0         -0.070642  28.322007  3.633306   
           ABT       6585900.0         -0.016917  48.111295  3.600365   
           ACGL      1284300.0         -0.001116  55.264187  2.968493   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

                      bbmid  ...  Return_1m  Return_3m  Return_6m  Return_9m  \
Date       Ticker            ...                                               
2015-02-09 A       3.600646  ...  -0.007626  -0.000597   0.005527   0.000829   
           AAPL    3.268119  ...   0.006642   0.001760   0.004268   0.010776   
           ABBV    3.719573  ...  -0.024956  -0.008447  -0.013925  -0.014198   
           ABT     3.629855  ...  -0.021539  -0.004820  -0.000896   0.002119   
           ACGL    2.985903  ...  -0.003999  -0.000446   0.005109  -0.000167   
...                     ...  ...        ...        ...        ...        ...   
2024-12-31 XYL     4.799889  ...  -0.002236  -0.006942  -0.001600  -0.004448   
           YUM     4.908393  ...   0.004793  -0.004530   0.002254  -0.000471   
           ZBH     4.679910  ...   0.002182  -0.004315  -0.001958  -0.002275   
           ZBRA    5.989551  ...   0.006174  -0.009010  -0.002913  -0.002495   
           ZTS     5.140142  ...   0.004253  -0.005243  -0.001941  -0.005221   

                   Return_12m    Mkt-RF       SMB       HML       RMW  \
Date       Ticker                                                       
2015-02-09 A        -0.001291  0.007615 -0.057369  0.031811  0.046662   
           AAPL      0.005601  0.092282 -0.040413 -0.028914  0.156016   
           ABBV     -0.010511  0.050595  0.054892 -0.068936  0.147411   
           ABT       0.000056 -0.007784  0.020929 -0.008331  0.146824   
           ACGL      0.000140 -0.051943 -0.072530  0.170694 -0.076591   
...                       ...       ...       ...       ...       ...   
2024-12-31 XYL      -0.003544  0.137447  0.004861  0.027339  0.237713   
           YUM      -0.002734  0.086861  0.058779  0.013125  0.211814   
           ZBH      -0.000762  0.163961 -0.129049  0.037081  0.003908   
           ZBRA     -0.004488  0.073991  0.163371 -0.062295  0.135212   
           ZTS      -0.007734  0.073236  0.060556 -0.155374  0.187400   

                        C

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1209531 entries, (Timestamp('2015-02-09 00:00:00'), 'A') to (Timestamp('2024-12-31 00:00:00'), 'ZTS')
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   adj close         1209531 non-null  float64
 1   close             1209531 non-null  float64
 2   high              1209531 non-null  float64
 3   low               1209531 non-null  float64
 4   open              1209531 non-null  float64
 5   volume            1209531 non-null  float64
 6   garman_klass_vol  1209531 non-null  float64
 7   rsi               1209531 non-null  float64
 8   bblow             1209531 non-null  float64
 9   bbmid             1209531 non-null  float64
 10  bbhigh            1209531 non-null  float64
 11  atr               1209531 non-null  float64
 12  macd              1209531 non-null  float64
 13  dollar_volume     1209531 non-null  float64
 14  Return_1m         1209531 

In [72]:
data.drop('adj close', axis = 1)

close        high         low        open  \
Date       Ticker                                                   
2015-02-09 A        39.040001   39.430000   38.950001   39.230000   
           AAPL     29.930000   29.959999   29.607500   29.637501   
           ABBV     55.480000   56.349998   55.180000   55.669998   
           ABT      44.520000   45.209999   44.299999   45.209999   
           ACGL     19.923332   20.063334   19.903334   20.006666   
...                       ...         ...         ...         ...   
2024-12-31 XYL     116.019997  117.110001  115.570000  116.779999   
           YUM     134.160004  134.789993  133.250000  134.089996   
           ZBH     105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  387.410004  381.750000  383.420013   
           ZTS     162.929993  163.750000  161.869995  162.899994   

                        volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-02-09 A         3586100.0         -0.003156  48.440784  3.567578   
           AAPL    155559200.0         -0.004133  66.460269  3.186643   
           ABBV     22522000.0         -0.070642  28.322007  3.633306   
           ABT       6585900.0         -0.016917  48.111295  3.600365   
           ACGL      1284300.0         -0.001116  55.264187  2.968493   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929333  4.871477   
           ZBH        683300.0          0.000085  41.626049  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259963  5.068102   

                      bbmid    bbhigh  ...  Return_1m  Return_3m  Return_6m  \
Date       Ticker                      ...                                    
2015-02-09 A       3.600646  3.633714  ...  -0.007626  -0.000597   0.005527   
           AAPL    3.268119  3.349596  ...   0.006642   0.001760   0.004268   
           ABBV    3.719573  3.805839  ...  -0.024956  -0.008447  -0.013925   
           ABT     3.629855  3.659344  ...  -0.021539  -0.004820  -0.000896   
           ACGL    2.985903  3.003313  ...  -0.003999  -0.000446   0.005109   
...                     ...       ...  ...        ...        ...        ...   
2024-12-31 XYL     4.799889  4.873531  ...  -0.002236  -0.006942  -0.001600   
           YUM     4.908393  4.945308  ...   0.004793  -0.004530   0.002254   
           ZBH     4.679910  4.706731  ...   0.002182  -0.004315  -0.001958   
           ZBRA    5.989551  6.036588  ...   0.006174  -0.009010  -0.002913   
           ZTS     5.140142  5.212182  ...   0.004253  -0.005243  -0.001941   

                   Return_9m  Return_12m    Mkt-RF       SMB       HML  \
Date       Ticker                                                        
2015-02-09 A        0.000829   -0.001291  0.007615 -0.057369  0.031811   
           AAPL     0.010776    0.005601  0.092282 -0.040413 -0.028914   
           ABBV    -0.014198   -0.010511  0.050595  0.054892 -0.068936   
           ABT      0.002119    0.000056 -0.007784  0.020929 -0.008331   
           ACGL    -0.000167    0.000140 -0.051943 -0.072530  0.170694   
...                      ...         ...       ...       ...       ...   
2024-12-31 XYL     -0.004448   -0.003544  0.137447  0.004861  0.027339   
           YUM     -0.000471   -0.002734  0.086861  0.058779  0.013125   
           ZBH     -0.002275   -0.000762  0.163961 -0.129049  0.037081   
           ZBRA    -0.002495   -0.004488  0.073991  0.163371 -0.062295   
           ZTS     -0.005221   -0.007734  0.073236  0.060556 -0.155374   

                        RMW       CMA  
Date       Ticker                      
2015-02-09 A       0.046662 -0.008291  
           AAPL    0.156016  0.211361  
           ABBV    0.1474